# Chapter 6: Always Blocks and Processes

## Introduction

Always blocks are fundamental constructs in SystemVerilog that describe how hardware behaves over time. They define processes that execute continuously during simulation and represent different types of hardware structures. SystemVerilog provides three specialized always blocks that make design intent clearer and help avoid common modeling mistakes.

## Types of Always Blocks

SystemVerilog introduces three specialized always blocks:

- `always_comb` - For combinational logic
- `always_ff` - For sequential logic (flip-flops)
- `always_latch` - For latches

These replace the generic `always` block from Verilog and provide better checking and clearer intent.

### always_comb for Combinational Logic

The `always_comb` block is used to model combinational logic where outputs change immediately when inputs change.

#### Basic Syntax

```systemverilog
always_comb begin
    // Combinational logic statements
end
```

#### Key Features

- Automatically sensitive to all inputs (no sensitivity list needed)
- Executes immediately when any input changes
- Should not contain clocked logic or memory elements
- Helps catch incomplete sensitivity lists

#### Examples

##### Simple Multiplexer
```systemverilog
module mux2to1 (
    input  logic sel,
    input  logic a, b,
    output logic y
);

always_comb begin
    if (sel)
        y = b;
    else
        y = a;
end

endmodule
```

##### ALU Example
```systemverilog
module simple_alu (
    input  logic [3:0] a, b,
    input  logic [1:0] op,
    output logic [3:0] result,
    output logic       zero
);

always_comb begin
    case (op)
        2'b00: result = a + b;    // Add
        2'b01: result = a - b;    // Subtract
        2'b10: result = a & b;    // AND
        2'b11: result = a | b;    // OR
        default: result = 4'b0;
    endcase
    
    zero = (result == 4'b0);
end

endmodule
```

##### Priority Encoder
```systemverilog
module priority_encoder (
    input  logic [7:0] req,
    output logic [2:0] grant,
    output logic       valid
);

always_comb begin
    valid = |req;  // OR reduction - true if any bit set
    
    if (req[7])      grant = 3'd7;
    else if (req[6]) grant = 3'd6;
    else if (req[5]) grant = 3'd5;
    else if (req[4]) grant = 3'd4;
    else if (req[3]) grant = 3'd3;
    else if (req[2]) grant = 3'd2;
    else if (req[1]) grant = 3'd1;
    else if (req[0]) grant = 3'd0;
    else             grant = 3'd0;
end

endmodule
```

### always_ff for Sequential Logic

The `always_ff` block is used to model sequential logic elements like flip-flops and registers.

#### Basic Syntax

```systemverilog
always_ff @(posedge clk) begin
    // Sequential logic statements
end

// With reset
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        // Reset logic
    end else begin
        // Normal clocked logic
    end
end
```

#### Key Features

- Must have a clocking event in sensitivity list
- Models memory elements (flip-flops, registers)
- Can include asynchronous reset/set
- Should use non-blocking assignments (<=) for clocked logic

#### Examples

##### Simple D Flip-Flop
```systemverilog
module dff (
    input  logic clk,
    input  logic rst_n,
    input  logic d,
    output logic q
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        q <= 1'b0;
    else
        q <= d;
end

endmodule
```

##### Counter with Enable
```systemverilog
module counter (
    input  logic       clk,
    input  logic       rst_n,
    input  logic       enable,
    input  logic       load,
    input  logic [7:0] load_value,
    output logic [7:0] count
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        count <= 8'b0;
    end else if (load) begin
        count <= load_value;
    end else if (enable) begin
        count <= count + 1;
    end
    // If neither load nor enable, count maintains its value
end

endmodule
```

##### Shift Register
```systemverilog
module shift_register #(
    parameter WIDTH = 8
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic             shift_en,
    input  logic             serial_in,
    output logic             serial_out,
    output logic [WIDTH-1:0] parallel_out
);

logic [WIDTH-1:0] shift_reg;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        shift_reg <= '0;
    end else if (shift_en) begin
        shift_reg <= {shift_reg[WIDTH-2:0], serial_in};
    end
end

assign serial_out = shift_reg[WIDTH-1];
assign parallel_out = shift_reg;

endmodule
```

##### State Machine Example
```systemverilog
typedef enum logic [1:0] {
    IDLE = 2'b00,
    ACTIVE = 2'b01,
    WAIT = 2'b10,
    DONE = 2'b11
} state_t;

module fsm (
    input  logic   clk,
    input  logic   rst_n,
    input  logic   start,
    input  logic   ready,
    output logic   busy,
    output logic   complete
);

state_t current_state, next_state;

// State register
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        current_state <= IDLE;
    else
        current_state <= next_state;
end

// Next state logic (combinational)
always_comb begin
    case (current_state)
        IDLE: begin
            if (start)
                next_state = ACTIVE;
            else
                next_state = IDLE;
        end
        
        ACTIVE: begin
            if (ready)
                next_state = WAIT;
            else
                next_state = ACTIVE;
        end
        
        WAIT: begin
            next_state = DONE;
        end
        
        DONE: begin
            next_state = IDLE;
        end
        
        default: next_state = IDLE;
    endcase
end

// Output logic (combinational)
always_comb begin
    busy = (current_state != IDLE) && (current_state != DONE);
    complete = (current_state == DONE);
end

endmodule
```

### always_latch for Latches

The `always_latch` block is used to model transparent latches, though latches are generally discouraged in synchronous design.

#### Basic Syntax

```systemverilog
always_latch begin
    // Latch logic statements
end
```

#### Key Features

- Models level-sensitive storage elements
- Should be avoided in most synchronous designs
- Can cause timing issues and make verification difficult
- Sometimes used for specific analog or mixed-signal applications

#### Example

```systemverilog
module d_latch (
    input  logic enable,
    input  logic d,
    output logic q
);

always_latch begin
    if (enable)
        q = d;
    // When enable is low, q retains its value (latch behavior)
end

endmodule
```

### Blocking vs. Non-Blocking Assignments

Understanding the difference between blocking (=) and non-blocking (<=) assignments is crucial for proper hardware modeling.

#### Blocking Assignments (=)

- Execute immediately in sequence
- Used in combinational logic (`always_comb`)
- Model wire-like behavior
- Can create race conditions if misused

#### Non-Blocking Assignments (<=)

- Scheduled to execute at end of time step
- Used in sequential logic (`always_ff`)
- Model register-like behavior
- Prevent race conditions in clocked logic

#### Examples Comparing Both

##### Combinational Logic - Use Blocking (=)
```systemverilog
// Correct - using blocking assignments
always_comb begin
    temp = a & b;
    y = temp | c;
end

// Incorrect - non-blocking in combinational logic
always_comb begin
    temp <= a & b;  // Wrong!
    y <= temp | c;  // Wrong! - temp not updated yet
end
```

##### Sequential Logic - Use Non-Blocking (<=)
```systemverilog
// Correct - using non-blocking assignments
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Creates shift register
end

// Incorrect - blocking in sequential logic
always_ff @(posedge clk) begin
    q1 = d;
    q2 = q1;   // Both update to 'd' simultaneously - not a shift register!
end
```

### Best Practices Summary

1. **Combinational logic (`always_comb`)**: Use blocking assignments (=)
2. **Sequential logic (`always_ff`)**: Use non-blocking assignments (<=)
3. **Mixed assignments**: Never mix blocking and non-blocking in the same always block

### Race Conditions and Common Pitfalls

#### Race Condition Example

```systemverilog
// Problematic code - race condition
module race_example (
    input  logic clk,
    input  logic d,
    output logic q1, q2
);

// Two separate always blocks updating at same time
always_ff @(posedge clk) begin
    q1 <= d;
end

always_ff @(posedge clk) begin
    q2 <= q1;  // Race condition! Order of execution matters
end

endmodule
```

```systemverilog
// Better approach - combine into one always block
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Now guaranteed to work correctly
end
```

#### Incomplete Sensitivity Lists (Verilog issue fixed by always_comb)

```systemverilog
// Old Verilog style - error prone
always @(a, b) begin  // Forgot 'c' in sensitivity list!
    y = a & b;
    z = y | c;        // 'z' won't update when 'c' changes
end

// SystemVerilog solution - automatic sensitivity
always_comb begin
    y = a & b;
    z = y | c;        // Automatically sensitive to a, b, and c
end
```

#### Process Control and Advanced Topics

#### Multiple Clock Domains

```systemverilog
module dual_clock_design (
    input  logic clk1, clk2,
    input  logic rst_n,
    input  logic data_in,
    output logic data_out
);

logic ff1, ff2;

// Clock domain 1
always_ff @(posedge clk1 or negedge rst_n) begin
    if (!rst_n)
        ff1 <= 1'b0;
    else
        ff1 <= data_in;
end

// Clock domain 2
always_ff @(posedge clk2 or negedge rst_n) begin
    if (!rst_n)
        ff2 <= 1'b0;
    else
        ff2 <= ff1;  // Potential metastability issue!
end

assign data_out = ff2;

endmodule
```

#### Generate Blocks with Always Blocks

```systemverilog
module parameterized_register #(
    parameter WIDTH = 8,
    parameter STAGES = 4
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

logic [WIDTH-1:0] pipe_reg [STAGES-1:0];

genvar i;
generate
    for (i = 0; i < STAGES; i++) begin : pipe_stage
        always_ff @(posedge clk or negedge rst_n) begin
            if (!rst_n) begin
                pipe_reg[i] <= '0;
            end else begin
                if (i == 0)
                    pipe_reg[i] <= data_in;
                else
                    pipe_reg[i] <= pipe_reg[i-1];
            end
        end
    end
endgenerate

assign data_out = pipe_reg[STAGES-1];

endmodule
```

### Best Practices and Guidelines

#### Design Guidelines

1. **Use appropriate always block types**:
   - `always_comb` for combinational logic
   - `always_ff` for sequential logic
   - Avoid `always_latch` unless specifically needed

2. **Assignment types**:
   - Blocking (=) in `always_comb`
   - Non-blocking (<=) in `always_ff`
   - Never mix both types in the same block

3. **Reset strategy**:
   - Use asynchronous reset for critical paths
   - Consider synchronous reset for better timing
   - Initialize all registers consistently

4. **Avoid common mistakes**:
   - Incomplete case statements
   - Inferred latches from incomplete if-else chains
   - Multiple drivers to same signal

#### Code Style Example

```systemverilog
module good_style_example (
    input  logic       clk,
    input  logic       rst_n,
    input  logic [7:0] data_in,
    input  logic       enable,
    output logic [7:0] data_out,
    output logic       valid
);

// Sequential logic - use always_ff with non-blocking
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        data_out <= 8'b0;
        valid    <= 1'b0;
    end else if (enable) begin
        data_out <= data_in;
        valid    <= 1'b1;
    end else begin
        valid    <= 1'b0;
        // data_out retains its value
    end
end

endmodule
```

### Summary

Always blocks are the cornerstone of behavioral modeling in SystemVerilog. The three specialized types (`always_comb`, `always_ff`, `always_latch`) provide clear intent and help prevent common coding errors. Understanding when to use blocking vs. non-blocking assignments is crucial for creating hardware that behaves as intended. Following the best practices outlined in this chapter will lead to more reliable, synthesizable, and maintainable code.


### Key Takeaways

- Use `always_comb` for combinational logic with blocking assignments (=)
- Use `always_ff` for sequential logic with non-blocking assignments (<=)
- Avoid `always_latch` unless specifically required
- Be consistent with reset strategies
- Understand race conditions and how to avoid them
- Always consider the hardware implications of your code

# SystemVerilog Always Blocks and Processes - Example Index

## always_comb for Combinational Logic

#### Basic Logic Gates
Simple AND, OR, XOR, NAND gates demonstrating basic combinational behavior

In [6]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_1__basic_logic_gates/"
files = ["basic_logic_gates.sv", "basic_logic_gates_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// basic_logic_gates.sv
module basic_logic_gates (
    input  logic a,        // First input
    input  logic b,        // Second input
    output logic and_out,  // AND gate output
    output logic or_out,   // OR gate output  
    output logic xor_out,  // XOR gate output
    output logic nand_out  // NAND gate output
);

    // Combinational logic assignments
    assign and_out  = a & b;   // AND gate
    assign or_out   = a | b;   // OR gate
    assign xor_out  = a ^ b;   // XOR gate
    assign nand_out = ~(a & b); // NAND gate (inverted AND)

    // Display gate outputs whenever inputs change
    always_comb begin
        $display(
            "Inputs: a=%b, b=%b | Outputs: AND=%b, OR=%b, XOR=%b, NAND=%b", 
            a, b, and_out, or_out, xor_out, nand_out);
    end

endmodule
```

```systemverilog
// basic_logic_gates_testbench.sv
module logic_gates_testbench;

    // Test signals
    logic test_a, test_b;
    logic result_and, result_or, result_xor, result_nand;

    // Instantiate the design under test
    basic_logic_gates DUT (
        .a(test_a),
        .b(test_b),
        .and_out(result_and),
        .or_out(result_or),
        .xor_out(result_xor),
        .nand_out(result_nand)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("basic_logic_gates_testbench.vcd");
        $dumpvars(0, basic_logic_gates_testbench);
        
        $display("=== Basic Logic Gates Test ===");
        $display();
        
        // Test all input combinations (truth table)
        $display("Testing all 2-input combinations:");
        
        // Test case 1: 00
        test_a = 1'b0; test_b = 1'b0;
        #10;
        
        // Test case 2: 01  
        test_a = 1'b0; test_b = 1'b1;
        #10;
        
        // Test case 3: 10
        test_a = 1'b1; test_b = 1'b0;
        #10;
        
        // Test case 4: 11
        test_a = 1'b1; test_b = 1'b1;
        #10;
        
        $display();
        $display("=== Truth Table Complete ===");
        $display("AND: outputs 1 only when both inputs are 1");
        $display("OR:  outputs 1 when at least one input is 1");
        $display("XOR: outputs 1 when inputs are different");
        $display("NAND: outputs 0 only when both inputs are 1 (inverted AND)");
        
        $finish;
    end

endmodule
```

Verilator Simulation Output:
=== Basic Logic Gates Test ===

Testing all 2-input combinations:
Inputs: a=0, b=0 | Outputs: AND=0, OR=0, XOR=0, NAND=1
Inputs: a=0, b=1 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=0, b=1 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=0 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=0 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0

=== Truth Table Complete ===
AND: outputs 1 only when both inputs are 1
OR:  outputs 1 when at least one input is 1
XOR: outputs 1 when inputs are different
NAND: outputs 0 only when both inputs are 1 (inverted AND)
- basic_logic_gates_testbench.sv:52: Verilog $finish
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0
Process finished with return code: 0
Removing Chapter_6_examples/example_1__basic_logic_gates/obj_dir directory...
Chapter_6_examples/example_1__basic_logic_gates/obj_dir removed successfully.


0

#### Decoder (3-to-8)
Address decoder that converts 3-bit input to 8 one-hot outputs with enable

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_2__address_decoder/"
files = ["address_decoder_3to8.sv", "address_decoder_3to8_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// address_decoder_3to8.sv
module address_decoder_3to8 (
    input  logic [2:0] address,    // 3-bit address input (0-7)
    input  logic       enable,     // Active high enable signal
    output logic [7:0] decoded_out // 8-bit one-hot output
);

    // Address decoder logic using case statement
    always_comb begin
        if (enable) begin
            case (address)
                3'b000: decoded_out = 8'b00000001; // Address 0 -> bit 0
                3'b001: decoded_out = 8'b00000010; // Address 1 -> bit 1
                3'b010: decoded_out = 8'b00000100; // Address 2 -> bit 2
                3'b011: decoded_out = 8'b00001000; // Address 3 -> bit 3
                3'b100: decoded_out = 8'b00010000; // Address 4 -> bit 4
                3'b101: decoded_out = 8'b00100000; // Address 5 -> bit 5
                3'b110: decoded_out = 8'b01000000; // Address 6 -> bit 6
                3'b111: decoded_out = 8'b10000000; // Address 7 -> bit 7
                default: decoded_out = 8'b00000000; // Should never happen
            endcase
        end else begin
            // All outputs disabled when enable is low
            decoded_out = 8'b00000000; 
        end
    end

    // Display decoder operation
    always_comb begin
        if (enable) begin
            $display(
                "Address Decoder: Enable=%b, Address=%3b (%0d) -> %8b", 
                enable, address, address, decoded_out);
        end else begin
            $display(
                "Address Decoder: Enable=%b -> All outputs disabled", 
                enable);
        end
    end

endmodule
```

```systemverilog
// address_decoder_3to8_testbench.sv
module address_decoder_testbench;

    // Test signals
    logic [2:0] test_address;
    logic       test_enable;
    logic [7:0] decoded_outputs;

    // Instantiate the design under test
    address_decoder_3to8 DUT (
        .address(test_address),
        .enable(test_enable),
        .decoded_out(decoded_outputs)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("address_decoder_3to8_testbench.vcd");
        $dumpvars(0, address_decoder_3to8_testbench);
        
        $display("=== 3-to-8 Address Decoder Test ===");
        $display();
        
        // Test 1: Decoder disabled (enable = 0)
        $display("Test 1: Testing with Enable = 0 (decoder disabled)");
        test_enable = 1'b0;
        
        for (int i = 0; i < 8; i++) begin
            test_address = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 2: Decoder enabled, test all addresses
        $display("Test 2: Testing with Enable = 1 (decoder enabled)");
        test_enable = 1'b1;
        
        for (int i = 0; i < 8; i++) begin
            test_address = i[2:0];
            #10;
            
            // Verify one-hot encoding
            if ($countones(decoded_outputs) == 1) begin
                $display("Address %0d correctly decoded to one-hot", i);
            end else begin
                $display("ERROR: Address %0d not one-hot output!", i);
            end
        end
        
        $display();
        
        // Test 3: Enable/disable transitions
        $display("Test 3: Testing enable/disable transitions");
        test_address = 3'b101; // Address 5
        
        $display("Setting address to 5, then toggling enable...");
        test_enable = 1'b1;
        #10;
        test_enable = 1'b0;
        #10;
        test_enable = 1'b1;
        #10;
        
        $display();
        $display("=== Address Decoder Test Complete ===");
        $display("One-hot encoding: Only one output bit high per address");
        $display("Enable control: All outputs 0 when enable is low");
        $display("Address range: 3-bit input covers addresses 0-7");
        
        $finish;
    end

    // Monitor for any unexpected behavior
    always @(decoded_outputs) begin
        if (test_enable && ($countones(decoded_outputs) != 1)) begin
            $display("WARNING: Non one-hot output detected when enabled!");
        end
    end

endmodule
```

Verilator Simulation Output:
=== 3-to-8 Address Decoder Test ===

Test 1: Testing with Enable = 0 (decoder disabled)
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled

Test 2: Testing with Enable = 1 (decoder enabled)
Address Decoder: Enable=1, Address=000 (0) -> 00000001
Address Decoder: Enable=1, 

0

#### Encoder (8-to-3 Binary)
Priority encoder that converts 8-bit input to 3-bit binary output

In [4]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_3__priority_encoder/"
files = ["priority_encoder_8to3.sv", "priority_encoder_8to3_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// priority_encoder_8to3.sv
module priority_encoder_8to3 (
    input  logic [7:0] data_in,     // 8-bit input data
    output logic [2:0] encoded_out, // 3-bit encoded output
    output logic       valid_out    // Valid signal (any input active)
);

    // Priority encoder logic - highest bit wins
    always_comb begin
        if (data_in[7]) begin
            encoded_out = 3'b111;       // Input 7 has highest priority
            valid_out = 1'b1;
        end else if (data_in[6]) begin
            encoded_out = 3'b110;       // Input 6
            valid_out = 1'b1;
        end else if (data_in[5]) begin
            encoded_out = 3'b101;       // Input 5
            valid_out = 1'b1;
        end else if (data_in[4]) begin
            encoded_out = 3'b100;       // Input 4
            valid_out = 1'b1;
        end else if (data_in[3]) begin
            encoded_out = 3'b011;       // Input 3
            valid_out = 1'b1;
        end else if (data_in[2]) begin
            encoded_out = 3'b010;       // Input 2
            valid_out = 1'b1;
        end else if (data_in[1]) begin
            encoded_out = 3'b001;       // Input 1
            valid_out = 1'b1;
        end else if (data_in[0]) begin
            encoded_out = 3'b000;       // Input 0 (lowest priority)
            valid_out = 1'b1;
        end else begin
            encoded_out = 3'b000;       // No inputs active
            valid_out = 1'b0;
        end
    end

    // Display encoder operation
    always_comb begin
        if (valid_out) begin
            $display("Priority Encoder: Input=%8b -> Encoded=%3b (%0d), Valid=%b", 
                     data_in, encoded_out, encoded_out, valid_out);
        end else begin
            $display("Priority Encoder: Input=%8b -> No valid input (Valid=%b)", 
                     data_in, valid_out);
        end
    end

endmodule
```

```systemverilog
// priority_encoder_8to3_testbench.sv
module priority_encoder_testbench;

    // Test signals
    logic [7:0] test_data_in;
    logic [2:0] encoded_result;
    logic       valid_result;

    // Instantiate the design under test
    priority_encoder_8to3 DUT (
        .data_in(test_data_in),
        .encoded_out(encoded_result),
        .valid_out(valid_result)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("priority_encoder_testbench.vcd");
        $dumpvars(0, priority_encoder_testbench);
        
        $display("=== 8-to-3 Priority Encoder Test ===");
        $display();
        
        // Test 1: No inputs active
        $display("Test 1: No inputs active");
        test_data_in = 8'b00000000;
        #10;
        
        $display();
        
        // Test 2: Single bit tests (test each priority level)
        $display("Test 2: Testing each input individually");
        for (int i = 0; i < 8; i++) begin
            test_data_in = 8'b00000001 << i;
            #10;
            
            // Verify correct encoding
            if (valid_result && (encoded_result == i[2:0])) begin
                $display("Input bit %0d correctly encoded", i);
            end else begin
                $display("ERROR: Input bit %0d encoding failed!", i);
            end
        end
        
        $display();
        
        // Test 3: Priority tests (multiple bits active)
        $display("Test 3: Testing priority behavior");
        
        // High priority wins over low priority
        test_data_in = 8'b10000001; // Bit 7 and 0 active
        #10;
        if (encoded_result == 3'b111) begin
            $display("Bit 7 correctly wins over bit 0");
        end else begin
            $display("ERROR: Priority not working correctly!");
        end
        
        // Middle priority test
        test_data_in = 8'b00101010; // Bits 5, 3, 1 active
        #10;
        if (encoded_result == 3'b101) begin
            $display("Bit 5 correctly wins over bits 3,1");
        end else begin
            $display("ERROR: Middle priority test failed!");
        end
        
        // All bits active (highest should win)
        test_data_in = 8'b11111111;
        #10;
        if (encoded_result == 3'b111) begin
            $display("Bit 7 correctly wins when all bits active");
        end else begin
            $display("ERROR: All bits test failed!");
        end
        
        $display();
        
        // Test 4: Random priority patterns
        $display("Test 4: Testing various priority patterns");
        
        test_data_in = 8'b01100000; // Bits 6,5 active -> 6 wins
        #10;
        
        test_data_in = 8'b00011100; // Bits 4,3,2 active -> 4 wins
        #10;
        
        test_data_in = 8'b00000110; // Bits 2,1 active -> 2 wins
        #10;
        
        $display();
        $display("=== Priority Encoder Test Complete ===");
        $display("Priority order: Bit 7 (highest) down to Bit 0 (lowest)");
        $display("Valid signal: Indicates at least one input is active");
        $display("Encoding: Highest priority active input -> binary output");
        $display();
        
        $finish;
    end

    // Monitor for unexpected behavior
    always @(test_data_in) begin
        #1; // Small delay to let combinational logic settle
        if (test_data_in != 8'b00000000 && !valid_result) begin
            $display("WARNING: Valid should be high when inputs are active!");
        end
        if (test_data_in == 8'b00000000 && valid_result) begin
            $display("WARNING: Valid should be low when no inputs active!");
        end
    end

endmodule
```

Verilator Simulation Output:
=== 8-to-3 Priority Encoder Test ===

Test 1: No inputs active
Priority Encoder: Input=00000000 -> No valid input (Valid=0)

Test 2: Testing each input individually
Priority Encoder: Input=00000001 -> Encoded=000 (0), Valid=1
Priority Encoder: Input=00000001 -> Encoded=000 (0), Valid=1
Priority Encoder: Input=00000001 -> Encoded=000 (0), Valid=1
Priority Encoder: Input=00000001 -> Encoded=000 (0), Valid=1
Input bit 0 correctly encoded
Priority Encoder: Input=00000010 -> Encoded=001 (1), Valid=1
Priority Encoder: Input=00000010 -> Encoded=001 (1), Valid=1
Priority Encoder: Input=00000010 -> Encoded=001 (1), Valid=1
Priority Encoder: Input=00000010 -> Encoded=001 (1), Valid=1
Input bit 1 correctly encoded
Priority Encoder: Input=00000100 -> Encoded=010 (2), Valid=1
Priority Encoder: Input=00000100 -> Encoded=010 (2), Valid=1
Priority Encoder: Input=00000100 -> Encoded=010 (2), Valid=1
Priority Encoder: Input=00000100 -> Encoded=010 (2), Valid=1
Input bit 2 co

0

#### Barrel Shifter
Combinational shifter that can shift/rotate data by variable amounts

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_4__combinational_shifter/"
files = ["combinational_shifter.sv", "combinational_shifter_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// combinational_shifter.sv
module combinational_shifter (
    input  logic [7:0] data_in,     // 8-bit input data
    input  logic [2:0] shift_amt,   // 3-bit shift amount (0-7)
    input  logic [1:0] shift_op,    // 2-bit operation select
    output logic [7:0] data_out     // 8-bit shifted output
);

    // Operation encoding:
    // 00: Logical shift left (LSL)
    // 01: Logical shift right (LSR)
    // 10: Arithmetic shift right (ASR)
    // 11: Rotate right (ROR)

    always_comb begin
        case (shift_op)
            2'b00: begin // Logical shift left
                data_out = data_in << shift_amt;
            end
            
            2'b01: begin // Logical shift right
                data_out = data_in >> shift_amt;
            end
            
            2'b10: begin // Arithmetic shift right (sign extend)
                data_out = $signed(data_in) >>> shift_amt;
            end
            
            2'b11: begin // Rotate right
                case (shift_amt)
                    3'd0: data_out = data_in;
                    3'd1: data_out = {data_in[0], data_in[7:1]};
                    3'd2: data_out = {data_in[1:0], data_in[7:2]};
                    3'd3: data_out = {data_in[2:0], data_in[7:3]};
                    3'd4: data_out = {data_in[3:0], data_in[7:4]};
                    3'd5: data_out = {data_in[4:0], data_in[7:5]};
                    3'd6: data_out = {data_in[5:0], data_in[7:6]};
                    3'd7: data_out = {data_in[6:0], data_in[7]};
                endcase
            end
            
            default: data_out = data_in; // Pass through
        endcase
    end

    // Display shifter operation
    always_comb begin
        case (shift_op)
            2'b00: $display("Shifter: LSL %8b << %0d = %8b", 
                           data_in, shift_amt, data_out);
            2'b01: $display("Shifter: LSR %8b >> %0d = %8b", 
                           data_in, shift_amt, data_out);
            2'b10: $display("Shifter: ASR %8b >>> %0d = %8b", 
                           data_in, shift_amt, data_out);
            2'b11: $display("Shifter: ROR %8b rotate %0d = %8b", 
                           data_in, shift_amt, data_out);
        endcase
    end

endmodule
```

```systemverilog
// combinational_shifter_testbench.sv
module shifter_testbench;

    // Test signals
    logic [7:0] test_data;
    logic [2:0] test_shift_amt;
    logic [1:0] test_shift_op;
    logic [7:0] shifted_result;

    // Operation names for display
    string op_names[4] = '{"LSL", "LSR", "ASR", "ROR"};

    // Instantiate the design under test
    combinational_shifter DUT (
        .data_in(test_data),
        .shift_amt(test_shift_amt),
        .shift_op(test_shift_op),
        .data_out(shifted_result)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("shifter_testbench.vcd");
        $dumpvars(0, shifter_testbench);
        
        $display("=== Combinational Shifter Test ===");
        $display();
        
        // Test 1: Logical shift left (LSL)
        $display("Test 1: Logical Shift Left (LSL)");
        test_data = 8'b10110011;
        test_shift_op = 2'b00; // LSL
        
        for (int i = 0; i <= 4; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 2: Logical shift right (LSR)
        $display("Test 2: Logical Shift Right (LSR)");
        test_data = 8'b10110011;
        test_shift_op = 2'b01; // LSR
        
        for (int i = 0; i <= 4; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 3: Arithmetic shift right (ASR)
        $display("Test 3: Arithmetic Shift Right (ASR) - Negative number");
        test_data = 8'b11010110; // Negative in 2's complement
        test_shift_op = 2'b10; // ASR
        
        for (int i = 0; i <= 4; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 4: ASR with positive number
        $display("Test 4: Arithmetic Shift Right (ASR) - Positive number");
        test_data = 8'b01010110; // Positive number
        test_shift_op = 2'b10; // ASR
        
        for (int i = 0; i <= 3; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 5: Rotate right (ROR)
        $display("Test 5: Rotate Right (ROR)");
        test_data = 8'b10110001;
        test_shift_op = 2'b11; // ROR
        
        for (int i = 0; i <= 7; i++) begin
            test_shift_amt = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 6: Edge cases
        $display("Test 6: Edge cases");
        
        // All zeros
        test_data = 8'b00000000;
        test_shift_op = 2'b00; // LSL
        test_shift_amt = 3'd3;
        #10;
        
        // All ones
        test_data = 8'b11111111;
        test_shift_op = 2'b01; // LSR
        test_shift_amt = 3'd2;
        #10;
        
        // Maximum shift
        test_data = 8'b10101010;
        test_shift_op = 2'b00; // LSL
        test_shift_amt = 3'd7;
        #10;
        
        $display();
        $display("=== Shifter Test Complete ===");
        $display("LSL: Logical shift left - zeros fill from right");
        $display("LSR: Logical shift right - zeros fill from left");
        $display("ASR: Arithmetic shift right - sign bit extends");
        $display("ROR: Rotate right - bits wrap around");
        $display();
        
        $finish;
    end

    // Function to verify LSL operation
    function logic [7:0] expected_lsl(logic [7:0] data, logic [2:0] amt);
        return data << amt;
    endfunction

    // Function to verify LSR operation
    function logic [7:0] expected_lsr(logic [7:0] data, logic [2:0] amt);
        return data >> amt;
    endfunction

    // Function to verify ASR operation
    function logic [7:0] expected_asr(logic [7:0] data, logic [2:0] amt);
        return $signed(data) >>> amt;
    endfunction

    // Function to verify ROR operation
    function logic [7:0] expected_ror(logic [7:0] data, logic [2:0] amt);
        case (amt)
            3'd0: return data;
            3'd1: return {data[0], data[7:1]};
            3'd2: return {data[1:0], data[7:2]};
            3'd3: return {data[2:0], data[7:3]};
            3'd4: return {data[3:0], data[7:4]};
            3'd5: return {data[4:0], data[7:5]};
            3'd6: return {data[5:0], data[7:6]};
            3'd7: return {data[6:0], data[7]};
        endcase
    endfunction

    // Monitor and verify all operations
    always @(shifted_result) begin
        #1; // Small delay for display
        case (test_shift_op)
            2'b00: begin // LSL verification
                if (shifted_result == expected_lsl(test_data, test_shift_amt)) begin
                    $display("LSL verification passed");
                end else begin
                    $display(
                        "LSL verification failed! Expected: %8b, Got: %8b", 
                        expected_lsl(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            2'b01: begin // LSR verification
                if (shifted_result == expected_lsr(test_data, test_shift_amt)) begin
                    $display("LSR verification passed");
                end else begin
                    $display(
                        "LSR verification failed! Expected: %8b, Got: %8b", 
                        expected_lsr(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            2'b10: begin // ASR verification
                if (shifted_result == expected_asr(test_data, test_shift_amt)) begin
                    $display("ASR verification passed");
                end else begin
                    $display(
                        "ASR verification failed! Expected: %8b, Got: %8b", 
                        expected_asr(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            2'b11: begin // ROR verification
                if (shifted_result == expected_ror(test_data, test_shift_amt)) begin
                    $display("ROR verification passed");
                end else begin
                    $display(
                        "ROR verification failed! Expected: %8b, Got: %8b", 
                        expected_ror(test_data, test_shift_amt),
                        shifted_result);
                end
            end
            default: begin
                $display("Unknown operation: %2b", test_shift_op);
            end
        endcase
    end

endmodule
```

Verilator Simulation Output:
=== Combinational Shifter Test ===

Test 1: Logical Shift Left (LSL)
Shifter: LSL 10110011 << 0 = 10110011
Shifter: LSL 10110011 << 1 = 01100110
Shifter: LSL 10110011 << 1 = 01100110
LSL verification passed
Shifter: LSL 10110011 << 1 = 01100110
Shifter: LSL 10110011 << 1 = 01100110
Shifter: LSL 10110011 << 2 = 11001100
Shifter: LSL 10110011 << 2 = 11001100
LSL verification passed
Shifter: LSL 10110011 << 2 = 11001100
Shifter: LSL 10110011 << 2 = 11001100
Shifter: LSL 10110011 << 3 = 10011000
Shifter: LSL 10110011 << 3 = 10011000
LSL verification passed
Shifter: LSL 10110011 << 3 = 10011000
Shifter: LSL 10110011 << 3 = 10011000
Shifter: LSL 10110011 << 4 = 00110000
Shifter: LSL 10110011 << 4 = 00110000
LSL verification passed
Shifter: LSL 10110011 << 4 = 00110000
Shifter: LSL 10110011 << 4 = 00110000

Test 2: Logical Shift Right (LSR)
Shifter: LSR 10110011 >> 0 = 10110011
Shifter: LSR 10110011 >> 0 = 10110011
LSR verification passed
Shifter: LSR 10110011 >> 

0

#### Seven-Segment Display Driver
BCD to seven-segment decoder for numeric display applications

In [9]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_5__bcd_seven_segment/"
files = ["bcd_seven_segment.sv", "bcd_seven_segment_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// bcd_seven_segment.sv
// BCD to Seven-Segment Display Decoder
// Converts 4-bit BCD input (0-9) to 7-segment display pattern

module bcd_seven_segment (
    input  logic [3:0] bcd_in,      // 4-bit BCD input (0-9)
    output logic [6:0] segments     // 7-segment output [a,b,c,d,e,f,g]
);

    // Seven-segment display layout:
    //  aaa
    // f   b
    // f   b
    //  ggg
    // e   c
    // e   c
    //  ddd
    
    // segments[6] = a, segments[5] = b, segments[4] = c, segments[3] = d
    // segments[2] = e, segments[1] = f, segments[0] = g
    // Active HIGH (1 = segment ON, 0 = segment OFF)

    always_comb begin
        case (bcd_in)
            4'h0: segments = 7'b1111110; // Display "0" - all except g
            4'h1: segments = 7'b0110000; // Display "1" - b,c only
            4'h2: segments = 7'b1101101; // Display "2" - a,b,d,e,g
            4'h3: segments = 7'b1111001; // Display "3" - a,b,c,d,g
            4'h4: segments = 7'b0110011; // Display "4" - b,c,f,g
            4'h5: segments = 7'b1011011; // Display "5" - a,c,d,f,g
            4'h6: segments = 7'b1011111; // Display "6" - a,c,d,e,f,g
            4'h7: segments = 7'b1110000; // Display "7" - a,b,c
            4'h8: segments = 7'b1111111; // Display "8" - all segments
            4'h9: segments = 7'b1111011; // Display "9" - a,b,c,d,f,g
            default: segments = 7'b0000000; // Blank display for invalid BCD
        endcase
    end

    // Display decoder operation for simulation
    always_comb begin
        case (bcd_in)
            4'h0: $display(
                "BCD Decoder: %h -> '0' (segments: %7b)", bcd_in, segments);
            4'h1: $display(
                "BCD Decoder: %h -> '1' (segments: %7b)", bcd_in, segments);
            4'h2: $display(
                "BCD Decoder: %h -> '2' (segments: %7b)", bcd_in, segments);
            4'h3: $display(
                "BCD Decoder: %h -> '3' (segments: %7b)", bcd_in, segments);
            4'h4: $display(
                "BCD Decoder: %h -> '4' (segments: %7b)", bcd_in, segments);
            4'h5: $display(
                "BCD Decoder: %h -> '5' (segments: %7b)", bcd_in, segments);
            4'h6: $display(
                "BCD Decoder: %h -> '6' (segments: %7b)", bcd_in, segments);
            4'h7: $display(
                "BCD Decoder: %h -> '7' (segments: %7b)", bcd_in, segments);
            4'h8: $display(
                "BCD Decoder: %h -> '8' (segments: %7b)", bcd_in, segments);
            4'h9: $display(
                "BCD Decoder: %h -> '9' (segments: %7b)", bcd_in, segments);
            default: $display(
                "BCD Decoder: %h -> BLANK (invalid BCD)", bcd_in);
        endcase
    end

endmodule
```

```systemverilog
// bcd_seven_segment_testbench.sv
module bcd_testbench;

    // Test signals
    logic [3:0] test_bcd;
    logic [6:0] display_segments;

    // Expected patterns for verification
    logic [6:0] expected_patterns [0:9] = '{
        7'b1111110, // 0
        7'b0110000, // 1
        7'b1101101, // 2
        7'b1111001, // 3
        7'b0110011, // 4
        7'b1011011, // 5
        7'b1011111, // 6
        7'b1110000, // 7
        7'b1111111, // 8
        7'b1111011  // 9
    };

    // Instantiate the design under test
    bcd_seven_segment DUT (
        .bcd_in(test_bcd),
        .segments(display_segments)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("bcd_testbench.vcd");
        $dumpvars(0, bcd_testbench);
        
        $display("=== BCD to Seven-Segment Decoder Test ===");
        $display("Segment order: [a,b,c,d,e,f,g] (MSB to LSB)");
        $display("Display layout:");
        $display("  aaa  ");
        $display(" f   b ");
        $display(" f   b ");
        $display("  ggg  ");
        $display(" e   c ");
        $display(" e   c ");
        $display("  ddd  ");
        $display();
        
        // Test all valid BCD inputs (0-9)
        $display("Testing valid BCD inputs (0-9):");
        for (int i = 0; i <= 9; i++) begin
            test_bcd = i[3:0];
            #10;
            
            // Verify the output
            if (display_segments == expected_patterns[i]) begin
                $display("âœ“ BCD %0d: PASS - Segments: %7b", i, display_segments);
            end else begin
                $display("âœ— BCD %0d: FAIL - Expected: %7b, Got: %7b", 
                        i, expected_patterns[i], display_segments);
            end
        end
        
        $display();
        
        // Test invalid BCD inputs (10-15)
        $display("Testing invalid BCD inputs (A-F):");
        for (int i = 10; i <= 15; i++) begin
            test_bcd = i[3:0];
            #10;
            
            if (display_segments == 7'b0000000) begin
                $display("âœ“ BCD %X: PASS - Blank display (segments: %7b)", i, display_segments);
            end else begin
                $display("âœ— BCD %X: FAIL - Expected blank, Got: %7b", i, display_segments);
            end
        end
        
        $display();
        
        // Visual representation test
        $display("Visual representation of each digit:");
        for (int i = 0; i <= 9; i++) begin
            test_bcd = i[3:0];
            #10;
            $display("Digit %0d:", i);
            display_seven_segment(display_segments);
            $display();
        end
        
        $display("=== BCD Decoder Test Complete ===");
        $finish;
    end

    // Function to display seven-segment pattern visually
    task display_seven_segment(logic [6:0] seg);
        string a, b, c, d, e, f, g;
        
        // Convert segment bits to display characters
        a = seg[6] ? "###" : "   ";
        b = seg[5] ? "#" : " ";
        c = seg[4] ? "#" : " ";
        d = seg[3] ? "###" : "   ";
        e = seg[2] ? "#" : " ";
        f = seg[1] ? "#" : " ";
        g = seg[0] ? "###" : "   ";
        
        // Display the seven-segment pattern
        $display(" %s ", a);
        $display("%s   %s", f, b);
        $display("%s   %s", f, b);
        $display(" %s ", g);
        $display("%s   %s", e, c);
        $display("%s   %s", e, c);
        $display(" %s ", d);
    endtask

    // Monitor for any changes
    always @(display_segments) begin
        #1; // Small delay for clean display
    end

endmodule
```

Verilator Simulation Output:
=== BCD to Seven-Segment Decoder Test ===
Segment order: [a,b,c,d,e,f,g] (MSB to LSB)
Display layout:
  aaa
 f   b
 f   b
  ggg
 e   c
 e   c
  ddd

Testing valid BCD inputs (0-9):
BCD Decoder: 0 -> '0' (segments: 1111110)
âœ“ BCD 0: PASS - Segments: 1111110
BCD Decoder: 1 -> '1' (segments: 0110000)
BCD Decoder: 1 -> '1' (segments: 0110000)
BCD Decoder: 1 -> '1' (segments: 0110000)
BCD Decoder: 1 -> '1' (segments: 0110000)
âœ“ BCD 1: PASS - Segments: 0110000
BCD Decoder: 2 -> '2' (segments: 1101101)
BCD Decoder: 2 -> '2' (segments: 1101101)
BCD Decoder: 2 -> '2' (segments: 1101101)
BCD Decoder: 2 -> '2' (segments: 1101101)
âœ“ BCD 2: PASS - Segments: 1101101
BCD Decoder: 3 -> '3' (segments: 1111001)
BCD Decoder: 3 -> '3' (segments: 1111001)
BCD Decoder: 3 -> '3' (segments: 1111001)
BCD Decoder: 3 -> '3' (segments: 1111001)
âœ“ BCD 3: PASS - Segments: 1111001
BCD Decoder: 4 -> '4' (segments: 0110011)
BCD Decoder: 4 -> '4' (segments: 0110011)
BCD Decoder: 4 -

0

#### Comparator Circuit
Multi-bit magnitude comparator with equal, greater-than, less-than outputs

In [ ]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_6__magnitude_comparator/"
files = ["magnitude_comparator.sva", "magnitude_comparator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// magnitude_comparator.sv
// Multi-bit Magnitude Comparator
// Compares two N-bit unsigned numbers and generates comparison outputs

module magnitude_comparator #(
    parameter WIDTH = 8  // Bit width of inputs (default 8-bit)
) (
    input  logic [WIDTH-1:0] a_in,    // First number input
    input  logic [WIDTH-1:0] b_in,    // Second number input
    output logic             equal,   // Output: a == b
    output logic             greater, // Output: a > b
    output logic             less     // Output: a < b
);

    // Combinational comparison logic
    always_comb begin
        equal   = (a_in == b_in);
        greater = (a_in > b_in);
        less    = (a_in < b_in);
    end

    // Display comparison results for simulation
    always_comb begin
        if (equal)
            $display("Comparator: %0d == %0d (Equal)", a_in, b_in);
        else if (greater)
            $display("Comparator: %0d > %0d (Greater)", a_in, b_in);
        else if (less)
            $display("Comparator: %0d < %0d (Less)", a_in, b_in);
    end

    // Verification: exactly one output should be high
    always_comb begin
        assert (equal + greater + less == 1) 
        else $error("Comparator error: Multiple or no outputs active!");
    end

endmodule
```

```systemverilog
// magnitude_comparator_testbench.sv
module comparator_testbench;

    // Test parameters
    parameter WIDTH = 8;
    
    // Test signals
    logic [WIDTH-1:0] test_a;
    logic [WIDTH-1:0] test_b;
    logic eq_out, gt_out, lt_out;
    
    // Test counters
    int test_count = 0;
    int pass_count = 0;
    int fail_count = 0;

    // Instantiate the design under test
    magnitude_comparator #(.WIDTH(WIDTH)) DUT (
        .a_in(test_a),
        .b_in(test_b),
        .equal(eq_out),
        .greater(gt_out),
        .less(lt_out)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("comparator_testbench.vcd");
        $dumpvars(0, comparator_testbench);
        
        $display("=== %0d-bit Magnitude Comparator Test ===", WIDTH);
        $display("Testing A vs B with Equal, Greater, Less outputs");
        $display();
        
        // Test 1: Equal values
        $display("Test 1: Equal Values");
        test_equal_values();
        $display();
        
        // Test 2: A greater than B
        $display("Test 2: A > B Cases");
        test_greater_than();
        $display();
        
        // Test 3: A less than B
        $display("Test 3: A < B Cases");
        test_less_than();
        $display();
        
        // Test 4: Edge cases
        $display("Test 4: Edge Cases");
        test_edge_cases();
        $display();
        
        // Test 5: Random comprehensive test
        $display("Test 5: Random Comprehensive Test");
        test_random_values();
        $display();
        
        // Final summary
        $display("=== Test Summary ===");
        $display("Total tests: %0d", test_count);
        $display("Passed: %0d", pass_count);
        $display("Failed: %0d", fail_count);
        if (fail_count == 0) begin
            $display("ALL TESTS PASSED!");
        end else begin
            $display("%0d TESTS FAILED", fail_count);
        end
        
        $finish;
    end

    // Task to test equal values
    task automatic test_equal_values();
        logic [WIDTH-1:0] test_values [5];
        
        // Initialize test values
        test_values[0] = 0;
        test_values[1] = 1;
        test_values[2] = 15;
        test_values[3] = 128;
        test_values[4] = 255;
        
        foreach (test_values[i]) begin
            test_a = test_values[i];
            test_b = test_values[i];
            #10;
            verify_result(1, 0, 0, "Equal");
        end
    endtask

    // Task to test A > B cases
    task automatic test_greater_than();
        // Test arrays for A > B cases
        logic [WIDTH-1:0] test_a_vals [5] = '{10, 255, 128, 100, 200};
        logic [WIDTH-1:0] test_b_vals [5] = '{5, 254, 127, 50, 199};
        
        for (int i = 0; i < 5; i++) begin
            test_a = test_a_vals[i];
            test_b = test_b_vals[i];
            #10;
            verify_result(0, 1, 0, "Greater");
        end
    endtask

    // Task to test A < B cases
    task automatic test_less_than();
        // Test arrays for A < B cases
        logic [WIDTH-1:0] test_a_vals [5] = '{5, 0, 127, 50, 199};
        logic [WIDTH-1:0] test_b_vals [5] = '{10, 1, 128, 100, 200};
        
        for (int i = 0; i < 5; i++) begin
            test_a = test_a_vals[i];
            test_b = test_b_vals[i];
            #10;
            verify_result(0, 0, 1, "Less");
        end
    endtask

    // Task to test edge cases
    task automatic test_edge_cases();
        // Minimum values
        test_a = 0; test_b = 0;
        #10; verify_result(1, 0, 0, "Min Equal");
        
        // Maximum values
        test_a = {WIDTH{1'b1}}; test_b = {WIDTH{1'b1}};
        #10; verify_result(1, 0, 0, "Max Equal");
        
        // Min vs Max
        test_a = 0; test_b = {WIDTH{1'b1}};
        #10; verify_result(0, 0, 1, "Min < Max");
        
        // Max vs Min
        test_a = {WIDTH{1'b1}}; test_b = 0;
        #10; verify_result(0, 1, 0, "Max > Min");
        
        // Adjacent values
        test_a = 100; test_b = 101;
        #10; verify_result(0, 0, 1, "Adjacent Less");
        
        test_a = 101; test_b = 100;
        #10; verify_result(0, 1, 0, "Adjacent Greater");
    endtask

    // Task for random testing
    task automatic test_random_values();
        logic [31:0] rand_val;
        
        for (int i = 0; i < 20; i++) begin
            // Generate random values with proper bit width
            rand_val = $random();
            test_a = rand_val[WIDTH-1:0];
            rand_val = $random();
            test_b = rand_val[WIDTH-1:0];
            #10;
            
            // Determine expected result
            if (test_a == test_b)
                verify_result(1, 0, 0, "Random Equal");
            else if (test_a > test_b)
                verify_result(0, 1, 0, "Random Greater");
            else
                verify_result(0, 0, 1, "Random Less");
        end
    endtask

    // Verification task
    task automatic verify_result(logic exp_eq, logic exp_gt, logic exp_lt, string test_name);
        test_count++;
        
        if (eq_out == exp_eq && gt_out == exp_gt && lt_out == exp_lt) begin
            pass_count++;
            $display(
                "%s: A=%0d, B=%0d -> EQ=%b GT=%b LT=%b", 
                test_name, test_a, test_b, eq_out, gt_out, lt_out);
        end else begin
            fail_count++;
            $display(
                "%s: A=%0d, B=%0d -> Expected: EQ=%b GT=%b LT=%b, Got: EQ=%b GT=%b LT=%b", 
                test_name, test_a, test_b, exp_eq, exp_gt, exp_lt, eq_out, gt_out, lt_out);
        end
    endtask

    // Monitor for timing analysis
    always @(test_a or test_b) begin
        #1; // Small delay to let outputs settle
        // Check that exactly one output is active
        if ((eq_out + gt_out + lt_out) != 1) begin
            $display(
                "WARNING: Invalid output state at A=%0d, B=%0d",
                test_a, test_b);
        end
    end

    // Performance analysis
    initial begin
        #1000; // Wait for all tests to complete
        $display();
        $display("=== Performance Analysis ===");
        $display("Comparator width: %0d bits", WIDTH);
        $display("Maximum input value: %0d", (2**WIDTH)-1);
        $display("Total possible comparisons: %0d", (2**WIDTH) * (2**WIDTH));
        $display("Tests performed: %0d (%.2f%% coverage)", 
                test_count,
                (real'(test_count) / real'((2**WIDTH) * (2**WIDTH))) * 100.0);
    end

endmodule
```

Verilator Simulation Output:
=== 8-bit Magnitude Comparator Test ===
Testing A vs B with Equal, Greater, Less outputs

Test 1: Equal Values
Comparator: 0 == 0 (Equal)
Comparator: 0 == 0 (Equal)
Comparator: 0 == 0 (Equal)
Equal: A=0, B=0 -> EQ=1 GT=0 LT=0
Comparator: 1 == 1 (Equal)
Comparator: 1 == 1 (Equal)
Comparator: 1 == 1 (Equal)
Comparator: 1 == 1 (Equal)
Equal: A=1, B=1 -> EQ=1 GT=0 LT=0
Comparator: 15 == 15 (Equal)
Comparator: 15 == 15 (Equal)
Comparator: 15 == 15 (Equal)
Comparator: 15 == 15 (Equal)
Equal: A=15, B=15 -> EQ=1 GT=0 LT=0
Comparator: 128 == 128 (Equal)
Comparator: 128 == 128 (Equal)
Comparator: 128 == 128 (Equal)
Comparator: 128 == 128 (Equal)
Equal: A=128, B=128 -> EQ=1 GT=0 LT=0
Comparator: 255 == 255 (Equal)
Comparator: 255 == 255 (Equal)
Comparator: 255 == 255 (Equal)
Comparator: 255 == 255 (Equal)
Equal: A=255, B=255 -> EQ=1 GT=0 LT=0

Test 2: A > B Cases
Comparator: 10 > 5 (Greater)
Comparator: 10 > 5 (Greater)
Comparator: 10 > 5 (Greater)
Comparator: 10 > 5 

0

## always_ff for Sequential Logic

#### Ring Counter
Circular shift register where output feeds back to input

In [23]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_7__circular_shift_register/"
files = [
    "circular_shift_register.sv",
    "circular_shift_register_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// circular_shift_register.sv
// Simple 4-bit Circular Shift Register
// The MSB output feeds back to become the LSB input

module circular_shift_register (
    input  logic clk,         // Clock signal
    input  logic reset,       // Reset signal (active high)
    input  logic load,        // Load enable signal
    input  logic [3:0] data,  // 4-bit data to load
    output logic [3:0] q      // 4-bit register output
);

    // On each clock edge, shift left and feed MSB back to LSB
    always_ff @(posedge clk or posedge reset) begin
        if (reset) begin
            q <= 4'b0000;  // Clear register on reset
        end else if (load) begin
            q <= data;     // Load new data when load is asserted
        end else begin
            // Circular shift: MSB goes to LSB, others shift left
            q <= {q[2:0], q[3]};
        end
    end

    // Display the shifting pattern for simulation
    always @(posedge clk) begin
        if (!reset && !load) begin
            $display("Time %0t: Shift Register = %b", $time, q);
        end
    end

endmodule
```

```systemverilog
// circular_shift_register_testbench.sv
// Simple testbench for circular shift register

module shift_register_testbench;

    // Test signals
    logic clk = 0;
    logic reset;
    logic load;
    logic [3:0] data;
    logic [3:0] q;

    // Clock generation - 10ns period
    always #5 clk = ~clk;

    // Instantiate the circular shift register
    circular_shift_register DUT (
        .clk(clk),
        .reset(reset),
        .load(load),
        .data(data),
        .q(q)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("shift_register_testbench.vcd");
        $dumpvars(0, shift_register_testbench);
        
        $display("=== Circular Shift Register Test ===");
        $display("Demonstrating 4-bit circular shift with feedback");
        $display();
        
        // Initialize signals
        reset = 1;
        load = 0;
        data = 4'b0000;
        
        // Wait a few clocks then release reset
        repeat(2) @(posedge clk);
        reset = 0;
        $display("Reset released");
        
        // Test 1: Load pattern 1001 and watch it shift
        $display("\nTest 1: Loading pattern 1001");
        @(posedge clk);
        load = 1;
        data = 4'b1001;
        @(posedge clk);
        load = 0;
        $display("Loaded: %b", q);
        
        // Watch it shift for 8 clock cycles (should return to original after 4)
        $display("\nShifting pattern (should repeat every 4 clocks):");
        for (int i = 0; i < 8; i++) begin
            @(posedge clk);
            $display("Clock %0d: %b", i+1, q);
        end
        
        // Test 2: Load different pattern 0110
        $display("\nTest 2: Loading pattern 0110");
        @(posedge clk);
        load = 1;
        data = 4'b0110;
        @(posedge clk);
        load = 0;
        $display("Loaded: %b", q);
        
        // Watch this pattern shift
        $display("\nShifting new pattern:");
        for (int i = 0; i < 6; i++) begin
            @(posedge clk);
            $display("Clock %0d: %b", i+1, q);
        end
        
        // Test 3: Single bit pattern to clearly show circular nature
        $display("\nTest 3: Single bit pattern 0001 (clearly shows circulation)");
        @(posedge clk);
        load = 1;
        data = 4'b0001;
        @(posedge clk);
        load = 0;
        $display("Loaded: %b", q);
        
        $display("\nWatching single bit circulate:");
        for (int i = 0; i < 8; i++) begin
            @(posedge clk);
            $display("Clock %0d: %b <- Bit position: %0d", i+1, q, 
                    (q == 4'b0010) ? 1 : 
                    (q == 4'b0100) ? 2 : 
                    (q == 4'b1000) ? 3 : 
                    (q == 4'b0001) ? 0 : -1);
        end
        
        $display("\n=== Test Complete ===");
        $display("Notice how the patterns repeat every 4 clock cycles");
        $display("This demonstrates the circular feedback from MSB to LSB");
        
        $finish;
    end

    // Monitor changes
    always @(q) begin
        if (!reset) begin
            $display("  Register changed to: %b (decimal: %0d)", q, q);
        end
    end

endmodule
```

Verilator Simulation Output:
=== Circular Shift Register Test ===
Demonstrating 4-bit circular shift with feedback

Reset released

Test 1: Loading pattern 1001
Time 15: Shift Register = 0000
  Register changed to: 1001 (decimal: 9)
Loaded: 1001

Shifting pattern (should repeat every 4 clocks):
Time 35: Shift Register = 1001
  Register changed to: 0011 (decimal: 3)
Clock 1: 0011
Time 45: Shift Register = 0011
  Register changed to: 0110 (decimal: 6)
Clock 2: 0110
Time 55: Shift Register = 0110
  Register changed to: 1100 (decimal: 12)
Clock 3: 1100
Time 65: Shift Register = 1100
  Register changed to: 1001 (decimal: 9)
Clock 4: 1001
Time 75: Shift Register = 1001
  Register changed to: 0011 (decimal: 3)
Clock 5: 0011
Time 85: Shift Register = 0011
  Register changed to: 0110 (decimal: 6)
Clock 6: 0110
Time 95: Shift Register = 0110
  Register changed to: 1100 (decimal: 12)
Clock 7: 1100
Time 105: Shift Register = 1100
  Register changed to: 1001 (decimal: 9)
Clock 8: 1001

Test 2: Load

0

#### LFSR (Linear Feedback Shift Register)
Pseudo-random number generator using XOR feedback taps

#### FIFO Buffer
First-in-first-out memory buffer with read/write pointers and status flags

#### PWM Generator
Pulse-width modulation generator with configurable duty cycle

#### Traffic Light Controller
State machine controlling traffic light sequences with timing

#### UART Transmitter
Serial data transmitter with start/stop bits and baud rate control

#### Memory Interface Controller
Controller for read/write operations to external memory with handshaking

## always_latch for Latches

#### Address Latch
Transparent latch for holding address during memory access cycles

#### Data Bus Latch
Bidirectional data latch for bus isolation and timing control

#### Clock Gating Latch
Level-sensitive latch used in clock gating circuits (with warnings about usage)

## Blocking vs. Non-Blocking Assignments

#### Pipeline Register Example
Multi-stage pipeline showing correct vs. incorrect assignment usage

#### Shift Register Comparison
Side-by-side comparison of blocking vs. non-blocking in shift registers

#### Combinational Chain Example
Multi-level combinational logic showing assignment timing effects

## Race Conditions and Common Pitfalls

#### Cross-Coupled Latches
Example showing race conditions in feedback systems

#### Clock Domain Crossing
Synchronizer circuits to safely cross clock domains

#### Multiple Driver Detection
Examples of inadvertent multiple drivers and resolution

## Process Control and Advanced Topics

#### Clock Divider Circuit
Frequency divider with even and odd division ratios

#### Metastability Synchronizer
Two-flip-flop synchronizer for asynchronous signal capture

#### Parameterized Delay Line
Configurable delay element using generate blocks

#### Handshake Protocol Controller
Ready/valid handshaking for data transfer protocols

#### Reset Synchronizer
Circuit for clean reset release across clock domains